In [1]:
import pandas as pd
df = pd.read_csv("genre_mapping_v3.csv")
df.head()

,id,song,artist,timestamp,isrc,release_date,tags


In [2]:
df = df[["id","artist","isrc","release_date","tags"]].rename(columns={"id":"TRACK_ID","artist":"ARTIST_ID","tags":"TAGS"})
df["PATH"] = df["TRACK_ID"]
df = df[["TRACK_ID","ARTIST_ID","isrc","PATH","release_date","TAGS"]]
df.head()

,TRACK_ID,ARTIST_ID,isrc,PATH,release_date,TAGS


In [31]:
import numpy as np
train, validation, test = np.split(df.sample(frac=1, random_state=123), [int(.8*len(df)), int(.9*len(df))])

In [46]:
classes = set(train["TAGS"].unique().tolist()) & set(validation["TAGS"].unique().tolist()) & set(test["TAGS"].unique().tolist())
len(classes)

33

In [49]:
# Deleted songs
print(len(train), len(train) - len(train[train["TAGS"].isin(classes)]))
print(len(validation), len(validation) - len(validation[validation["TAGS"].isin(classes)]))
print(len(test), len(test) - len(test[test["TAGS"].isin(classes)]))

15349 76
1919 3
1919 10


In [47]:
for df, name in [(train,'train'), (validation,'validation'), (test,'test')]:
    df[df["TAGS"].isin(tags)].to_csv(f"split/genres/{name}.tsv",sep="\t",index=False)

In [ ]:
>>> import pandas as pd
>>> import os
>>> df = pd.read_csv("genre_mapping_v3.csv")
>>> df[df["id"].isin(os.listdir('/home/jupyter/models/jamendo-mood/dataset-audios-npy/'))]
>>> ids = [id.replace(".npy",".m4a") for id in os.listdir('/home/jupyter/models/jamendo-mood/dataset-audios-npy/')]
>>> df[df["id"].isin(ids)]
